# グリッドサーチ with 交差検証

---

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

- wine 読んでデータとラベルに分けて学習用とテスト用に分ける

In [2]:
df = pd.read_csv('wine.csv')

In [3]:
data = df.drop('label',axis=1)
label = df['label']

data_train,data_test,label_train,label_test=train_test_split(data,label,test_size=0.3,random_state=0)

- SVM のパラメータ C と gamma の候補をリストアップする
> すみません、ちょっといろいろ試して、あたりをつけました・・・

In [4]:
C = [1e2,1e3,1e4,1e5,1e6]
G = [1e-9,1e-8,1e-7,1e-6,1e-5]

- 上のパラメータで総当たり（グリッドサーチ）に交差検証して良さげなのを見つける
> この時、当然ながら学習用データを使う

In [14]:
for c in C:
    for g in G:
        clf = SVC(C=c,gamma=g)
        score=cross_val_score(clf,data_train,label_train,cv=5)
        print('C:',c,'\tgamma:',g,'\tscore:',score.mean())
        

C: 100.0 	gamma: 1e-09 	score: 0.6523333333333332
C: 100.0 	gamma: 1e-08 	score: 0.6523333333333332
C: 100.0 	gamma: 1e-07 	score: 0.6773333333333333
C: 100.0 	gamma: 1e-06 	score: 0.7093333333333333
C: 100.0 	gamma: 1e-05 	score: 0.8140000000000001
C: 1000.0 	gamma: 1e-09 	score: 0.6523333333333332
C: 1000.0 	gamma: 1e-08 	score: 0.6766666666666667
C: 1000.0 	gamma: 1e-07 	score: 0.734
C: 1000.0 	gamma: 1e-06 	score: 0.8143333333333332
C: 1000.0 	gamma: 1e-05 	score: 0.8943333333333333
C: 10000.0 	gamma: 1e-09 	score: 0.6766666666666667
C: 10000.0 	gamma: 1e-08 	score: 0.6693333333333333
C: 10000.0 	gamma: 1e-07 	score: 0.8546666666666667
C: 10000.0 	gamma: 1e-06 	score: 0.9106666666666665
C: 10000.0 	gamma: 1e-05 	score: 0.9269999999999999
C: 100000.0 	gamma: 1e-09 	score: 0.6853333333333333
C: 100000.0 	gamma: 1e-08 	score: 0.8386666666666667
C: 100000.0 	gamma: 1e-07 	score: 0.9349999999999999
C: 100000.0 	gamma: 1e-06 	score: 0.9269999999999999
C: 100000.0 	gamma: 1e-05 	score: 0.

- 良さげなのを３つくらい見繕ってみる
> (1) C: 1000000.0 	gamma: 1e-07 	score: 0.9516666666666665<br>(2) C: 1000000.0 	gamma: 1e-06 	score: 0.9433333333333334<br>(3) C: 100000.0 	gamma: 1e-07 	score: 0.9349999999999999

- 良さげな３つで分類器作ってみる

In [10]:
clf1 = SVC(C=1e6,gamma=1e-7)
clf2 = SVC(C=1e6,gamma=1e-6)
clf3 = SVC(C=1e5,gamma=1e-7)

- これらをそれぞれ「学習用データ」で学習し「テストデータ」で正解率みます
- パラメータの選定にも学習にも「テストデータ」は使ってないので、フェアな評価になりますね :-)

In [11]:
clf1.fit(data_train,label_train)
score1 = clf1.score(data_test,label_test)

clf2.fit(data_train,label_train)
score2 = clf2.score(data_test,label_test)

clf3.fit(data_train,label_train)
score3 = clf3.score(data_test,label_test)

In [12]:
print('(1) C: 1e6\tgamma',1e-7,'\tscore',score1)
print('(2) C: 1e6\tgamma',1e-6,'\tscore',score2)
print('(3) C: 1e5\tgamma',1e-7,'\tscore',score3)

(1) C: 1e6	gamma 1e-07 	score 1.0
(2) C: 1e6	gamma 1e-06 	score 0.9629629629629629
(3) C: 1e5	gamma 1e-07 	score 0.9629629629629629


- テストデータって何個ある？

In [13]:
data_test.count()[0]

54

### （1）は全問正解、それ以外は２個間違ってるもよう
> (54-2)/54 = 0.9629629629629629
- まあ、いい感じにSVMのパラメータ選べたんじゃないか？
- この５つからさらに絞り込むとか、工夫の余地はまだありそうな気がする